# Telco Customer Churn Prediction Assignment Problem Statement - 7

#### Name: Yenduru Poojith Manideep
#### BITS ID: 2024ac05966@wilp.bits-pilani.ac.in

#### Objective: Predict customer churn for a telecommunications company using machine learning models

## Section 1: Import Libraries and Dataset

In [ ]:
# Importing Required Libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For advanced visualization
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import PowerTransformer as pt


# Setting visualization style
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

### Load the Dataset

**Justification:**

Loading the raw CSV using `pandas.read_csv` allows us to efficiently parse and inspect the data. Pandas makes it easy to spot data types, missing values, and enables immediate profiling for downstream preprocessing and analysis tasks.

In [ ]:
# Loading Telco Customer Churn Dataset
df = pd.read_csv('Telco-Dataset.csv')
print('Dataset Loaded. Shape of the Dataset is:', df.shape)

## Section 2: Data Visualization and Exploration

Exploratory Data Analysis (EDA) is a critical step in any machine learning project. It helps build intuition about the data's structure, distributions, imbalances, correlations, and potential issues. Visualization makes patterns and outliers visible, while summary statistics and info reveal hidden or non-obvious data problems.

#### 2.a) Sanity Check: Preview Data
**Justification:**
Previewing the first few rows ensures the dataset loaded as expected—columns, data types, and target variable are intact. This helps spot major import/formatting errors instantly.

In [ ]:
df.head(10)  # Show 10 rows for a richer preview

#### 2.b) Describe and Check Dataset Details

- **Metadata:**
    - Number of rows and columns
    - Datatype of each variable
    - Count of missing/null values (initial scan)
    - Statistical summary of numerical columns
- **Justification:**
    - `.info()` and `.shape` confirm both variable types and missingness.
    - `.describe()` summarizes numerical variables, including any anomalies (negative values, extreme outliers, or constant columns)

While there are other profiling tools (like `pandas_profiling` or `dtale`), these are preferred for transparency and assignment reproducibility.

In [ ]:
# Dataset metadata & null scan
print('Dataset shape:', df.shape)

print('\nColumn Types and Null Scan:')
print(df.info())

I am converting TotalCharges from object to numeric (float), as it's actually a numerical data. 

We are doing this before itself because we need to do correlation analysis for numerical data in EDA step itself

In [ ]:

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

print('\nStatistical Description (Numerical Columns):')
display(df.describe())

#### 2.c) Visual Exploration (Univariate and Bivariate Analysis)

We'll visualize both categorical and numerical variables to check distributions, imbalances, and relationships.

**Univariate Analysis:** For each variable, visualize its distribution/frequency.
- **Justification:**
    - `countplot` is most effective for categorical (showing counts/imbalances, e.g., Churn)
    - `histplot` (with `kde`) for numerical (to check for skew, outliers, or multimodal distributions)
    - Alternatives: `boxplot` or `violinplot` (good for spotting outliers, but less direct for proportions). Histograms allow density shapes to appear clearly and are preferred for initial numeric variable review.

**Bivariate Analysis:** Relationship between features (categorical/numeric) and target (`Churn`).
- **Justification:**
    - For each feature, seeing how distributions change across the churn target can highlight strong predictors/imbalances or the need for grouping/binarization.

These plots form the basis for later feature transformation and selection.

In [ ]:
# Churn value counts to check balance
plt.figure(figsize=(4,3))
sns.countplot(data=df, x='Churn')
plt.title('Churn Class Distribution')
plt.show()

# Distribution of numerical features
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
fig, axs = plt.subplots(1, 3, figsize=(18, 4))
for i, col in enumerate(num_cols):
    sns.histplot(df[col], bins=25, kde=True, ax=axs[i])
    axs[i].set_title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

# Distribution of categorical features
cat_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
            'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

plt.figure(figsize=(16, 16))
for i, col in enumerate(cat_cols, 1):
    plt.subplot(4, 4, i)
    sns.countplot(x=col, data=df, color='steelblue')
    plt.tight_layout()
plt.suptitle('Categorical Feature Distributions', y=1.02, fontsize=16)
plt.show()

#### 2.d) Correlational Analysis
- **Numerical attribute correlation:** Useful to spot redundancy and strong links which may impact model multicollinearity or suggest engineered features.
- **Method:** Pearson correlation matrix is computed only on numerical features. Visualized as a heatmap (easier to see patterns than numbers).
- **Justification:**
    - **Why Pearson?** For continuous variables it’s most common and interpretable. Alternatives such as Spearman or Kendall available for ordinal/monotonic relations but here not required at first.
    - **Limitations:** This matrix ignores categorical predictors (which might relate strongly to churn—next steps will address that using mutual information/chi-square, etc). For classical ML, analyzing both correlation *and* categorical-target relationships is best practice for feature selection.
    - **Alternative techniques:** For datasets with many categorical features (like this one), we could use Cramer's V, mutual information, or pairwise Chi-square for categorical features, but correlation heatmap is a required industry baseline.

In [ ]:
# Correlational matrix (numerical)
plt.figure(figsize=(6,4))
corr = df[['tenure', 'MonthlyCharges', 'TotalCharges']].corr(method='pearson')
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix (Numerical Features)')
plt.show()

##### Justification: Impact of Correlation Analysis on Feature Selection

**Will this affect feature selection?**

- Correlational analysis of *numerical* features helps identify redundant or highly correlated (multicollinear) variables. In linear models (like Logistic Regression), high correlation between predictors can undermine interpretability and model stability, so one of two highly correlated features may be dropped. However, tree-based and other nonlinear models are less sensitive.
- However, most features in this dataset are **categorical**; Pearson's r ignores these. To assess feature importance for categorical columns, one should use chi-square, mutual information, or Cramér's V as supplement.
- **Therefore:** While correlation heatmaps guide *numeric* feature selection, *categorical* feature selection requires additional measures. For full rigor, we should supplement correlation matrices with information-theoretic/association analyses for categorical features in later steps. This multi-pronged approach maximizes model reliability and explanatory power.

---
# Section 3: Data Pre-processing and Cleaning

Careful pre-processing is crucial for classical machine learning models, as they have limited tolerance for data and encoding issues compared to deep learning. Here we address:
- Missing/null values and type inconsistencies
- Outlier and skew handling
- Encoding categorical variables: binary, ordinal, nominal (one-hot), with justification for each feature
- Feature scaling (Standardization/Normalization), with method selection rationale
- Identifying important features for prediction

## 3.a) Handling Missing/NULL Values and Data Type Consistency
#### Justification:
- Classical ML models like Logistic Regression, Decision Tree, KNN all require true numerical input; string/object types and NaNs must be fixed (impute or drop, but justify when dropping is acceptable vs business impact).
- For this dataset, `TotalCharges` sometimes encodes missing values as blanks (''), so we must coerce and fix these. Other columns with `No internet service`/`No phone service` are valid categories, not missing values, so should be left as-is for categorical encoding.
- Missing in categorical: impute with mode for non-informative missing; for numerical, mean/median depending on skew (see below).

In [ ]:
# Scan for missing values and datatypes
print('Missing values per column:')
print(df.isnull().sum())

# Special handling: Convert 'TotalCharges' to numeric; blanks to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Now, recalculate missing after conversion
print('Missing after TotalCharges coercion:', df['TotalCharges'].isnull().sum())
# Show rows with missing TotalCharges for manual inspection
display(df[df['TotalCharges'].isnull()])

# Safe to impute these as zero tenure means new customers with no charges
df['TotalCharges'] = df['TotalCharges'].fillna(0)

## 3.b) Outlier and Skewness Detection & Handling
#### Justification:
- Outliers can bias classical models, especially those using linear decision boundaries (e.g., Logistic Regression, KNN with distance).
- For tenure and charges, we'll use boxplots and statistical summaries to flag extreme values, then visually check distributions with log-transform or quantile capping if necessary (justified below).
- Feature distribution skewness matters for models assuming feature Gaussianity (e.g., KNN, Logistic Regression, some distance/variance-based metrics). We'll check for strong skew in numeric columns: if present, log transformation or scaling can help.

In [ ]:
# Visual inspection for outliers
fig, axs = plt.subplots(1, 3, figsize=(18,4))
for i, col in enumerate(['tenure', 'MonthlyCharges', 'TotalCharges']):
    sns.boxplot(x=df[col], ax=axs[i], color='lightcoral')
    axs[i].set_title(f'Boxplot - {col}')
plt.tight_layout()
plt.show()

# Skewness check
skew_vals = df[['tenure','MonthlyCharges','TotalCharges']].skew()
print('Skewness of Numeric Features:')
print(skew_vals)

# Why not immediately remove outliers? For customer churn, outliers are likely legitimate (very high tenure/charges are loyal customers), so removal may lose valuable business insight. We'll scale these instead rather than cap/drop.

## 3.c) Feature Transformation (Scaling/Standardization/Normalization)


##### We need to do Log1p or yeo-johnson Transformation on TotalCharges

Based on the histograms and skewness values:
- **Tenure**: Skewness is approximately 0.24 (near zero), indicating a roughly symmetric distribution. No transformation needed, as it's already suitable for models assuming normality.
- **MonthlyCharges**: Skewness is approximately -0.22 (slightly negative but close to zero), with a bimodal but balanced shape. Transformation isn't necessary, as the distribution doesn't exhibit extreme skew that would bias models.
- **TotalCharges**: Skewness is highly positive (approximately 0.96), with a long right tail due to accumulated charges over time and zeros for new customers. This skew can distort distance-based models (e.g., KNN) and linear assumptions in Logistic Regression.

**Why only TotalCharges?** The other numerical features ("tenure" and "MonthlyCharges") have skewness values close to zero and distributions that are not heavily tailed, so transforming them could unnecessarily alter their natural scale without providing benefits. Applying transformations universally might introduce artifacts or reduce interpretability without justification.

**Why yeo-johnson specifically?** Log transformations are standard for positively skewed data, as they compress the long tail and make the distribution more normal-like. We use yeo-johnson instead of a plain log because "TotalCharges" includes zero values (for new customers with no charges yet), and log(0) is undefined. yeo-johnson handles zeros gracefully while stabilizing variance and improving model fit.

In [ ]:
# Drop customerID
df_ml = df.drop('customerID', axis=1)

# Recode redundant categories for internet-dependent features
internet_dependent = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in internet_dependent:
    df_ml[col] = df_ml[col].replace('No internet service', 'No')

# Similarly for MultipleLines
df_ml['MultipleLines'] = df_ml['MultipleLines'].replace('No phone service', 'No')

pt = pt(method="yeo-johnson")
df_ml['TotalCharges'] = pt.fit_transform(df_ml[["TotalCharges"]])

# Verify skewness after transformation
print('Skewness after yeo-johnson on TotalCharges:', df_ml['TotalCharges'].skew())

# Optional: Re-plot histogram for TotalCharges to visualize improvement
plt.figure(figsize=(6,4))
sns.histplot(df_ml['TotalCharges'], kde=True)
plt.title('Distribution of TotalCharges after log1p')
plt.show()



#### Justification:
- **Why scale?** Classical ML models such as Logistic Regression (gradient-based), KNN (distance-based), and even tree ensembles (split-based, but still benefit) are sensitive to feature scale/variance. If numerical features vary on different scales, model could get biased or numerically unstable.
- **Which method?**
    - **StandardScaler (mean 0, std 1):** Best for features with approximately bell-shaped (normal) distributions, and for algorithms making use of means/stds or distance (KNN, LR).
    - **MinMaxScaler (0-1 scaling):** Used when features show non-Gaussian/skewed distributions. Keeps ranges between 0 and 1 (usually for algorithms relying on distance or bounded inputs).
    - **RobustScaler:** For numeric features with a lot of outliers or heavy skew, scales by median and IQR efficiently.

We'll use StandardScaler for 'tenure', 'MonthlyCharges', 'TotalCharges' as they are wide-range continuous and only mildly skewed, but document the reasoning for other methods.

In [ ]:
from sklearn.preprocessing import StandardScaler
# Instantiate scaler
scaler = StandardScaler()
num_cols = ['tenure','MonthlyCharges','TotalCharges']
df_ml[num_cols] = scaler.fit_transform(df_ml[num_cols])

# Verify scaling
df_ml[num_cols].describe().T[['mean','std']]

## 3.d) Encoding Categorical Features
#### Justification:
For classical ML, all input features must be numerical. But how we transform categorical variables greatly affects model results, interpretability, and risks of data leakage.

**Encoding options include:**
- **Label Encoding:** Assigns unique integer to each category. Good only for ordinal features (if order is meaningful) to preserve rank. *Problem:* Can create artificial order when applied to nominal features (e.g., gender).
- **One-Hot Encoding:** Dummy variables for each category (except one as reference). Essential for truly nominal (unordered) features since it avoids false ordinality.
- **Binary Encoding:** For high-cardinality features (many unique categories), it encodes categories as binary digits, reducing dimensionality compared to one-hot, but less interpretable. Not needed here due to low cardinality.

#### Assignment of encoding method for each feature:
| Feature             | Encoder     | Justification                                                      |
|---------------------|-------------|--------------------------------------------------------------------|
| gender              | Binary/One-hot      | Two categories, no order: one-hot preferred for LR/Tree/Ensemble |
| SeniorCitizen       | Numeric (0/1) | Already numeric (0,1)                                             |
| Partner, Dependents | Binary/One-hot      | Two categories, no order, one-hot to avoid LR bias              |
| PhoneService        | Binary/One-hot      | Two categories, no order, one-hot                                |
| MultipleLines       | One-hot      | 3 values: Yes/No/No phone service, one-hot preserves all info     |
| InternetService     | One-hot      | 3 nominal values                                                  |
| OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies | One-hot | 3 levels including 'No internet service', one-hot most robust |
| Contract            | One-hot      | 3 unordered types                                                 |
| PaperlessBilling    | Binary/One-hot      | Two values, no order, one-hot to avoid coefficients bias in LR  |
| PaymentMethod       | One-hot      | 4 different methods, no order, one-hot for model reliability      |

We avoid Label Encoding for non-ordinal variables to prevent giving unintended priority/order to categories, especially since Logistic Regression and tree methods handle one-hots natively.

**Note:** We'll drop the customerID (non-predictive identifier) before modeling.

**Churn** will be mapped to 1/0 for modeling.

In [ ]:

# Encode churn label
df_ml['Churn'] = df_ml['Churn'].map({'Yes': 1, 'No': 0})

# List categorical features (excluding target)
cat_feats = [col for col in df_ml.columns if df_ml[col].dtype == 'object' and col != 'Churn']

# Show unique values for categorical features (documentation)
for col in cat_feats:
    print(f"{col}: {df_ml[col].unique()}")

# Apply one-hot encoding (assign drop_first=False so all categories explicit)
df_ml = pd.get_dummies(df_ml, columns=cat_feats, drop_first=False)

print('Shape after encoding:', df_ml.shape)
df_ml.head()

## 3.e) Feature Importance Analysis for Engineering

**Why important:** In classical ML, removing uninformative or redundant features improves model generalization and speeds up training. There are several approaches:
- **Mutual Information:** Measures non-linear dependencies between categorical input and categorical target ('Churn').
- **SelectKBest with chi2:** Used when all values positive (e.g. for some categorical transformations; here, with one-hot encoding, this is valid since all non-neg). 
- **Tree feature importance:** Based on reduction in impurity, useful in tree models, less for linear/logistic.

We'll use mutual information for classification, as it is generic and robust for mixed-data types. This can guide optional feature pruning or highlight engineered interaction features in future.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

feat_importance = pd.Series(mi_scores, X.columns).sort_values(ascending=False)

# Top 15 features
feat_importance[:15].plot(kind='barh', figsize=(8,6))
plt.gca().invert_yaxis()
plt.title('Feature Importance (Mutual Information w/ Churn)')
plt.show()

# Display top features (table)
display(feat_importance.head(15).to_frame('MI Score'))

# Drop low-importance features (MI <= 0.01, keep important only)
selected_feats = feat_importance[feat_importance > 0.01].index.tolist()
if 'Churn' not in selected_feats:
    selected_feats.append('Churn')
df_ml = df_ml[selected_feats]


### Justification: Final State for Modeling
- **All features now numeric with appropriate encoding.**
- **Missing values and scale issues resolved.**
- **Feature importances extracted for further model interpretation.**
- **Outliers not removed due to business context; classical scalers used instead.**
- **Ready for robust model building in next section.**

---
# Section 4: Model Building

### Overview:
In this section, we develop and evaluate several classical machine learning models—Logistic Regression, Decision Tree, K-Nearest Neighbors, and an Ensemble (Random Forest)—to classify customer churn. We demonstrate the effect of different train/test splits and use cross-validation for hyperparameter tuning. For each model and step, we add justifications per the assignment instruction.


## 4.a) Train-Test Split (with multiple ratios)
#### Justification:
- The standard split for model validation is 80% train, 20% test. This provides enough data for training while holding out a portion for an unbiased evaluation. However, smaller or larger splits (such as 70/30, 90/10) can be tested for sensitivity.
- **80/20**: Widely accepted in industry; balances model learning and robust validation.
- **70/30**: More data for validation; may benefit overfit-prone models.
- **90/10**: More data for training; fewer for test—sometimes useful with smaller datasets or when model learning is challenging.
- **Justification:** Multiple splits are compared to ensure model results are not sensitive to choice of validation ratio. Results can be compared to check for split variance.

In [ ]:
from sklearn.model_selection import train_test_split
# Data for modeling
X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']

# 80/20 split (default)
X_train_80, X_test_20, y_train_80, y_test_20 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"80/20 split - Train: {X_train_80.shape}, Test: {X_test_20.shape}")

# 70/30 split
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"70/30 split - Train: {X_train_70.shape}, Test: {X_test_30.shape}")

# 90/10 split
X_train_90, X_test_10, y_train_90, y_test_10 = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
print(f"90/10 split - Train: {X_train_90.shape}, Test: {X_test_10.shape}")

### Data/Target distribution check (Sanity)
- Confirm that class balances are preserved using stratified split.
- Ensures valid model comparison across splits.

In [ ]:
print('Churn distribution in full set:', y.mean().round(3))
print('Churn in 80% train:', y_train_80.mean().round(3))
print('Churn in 20% test:', y_test_20.mean().round(3))
# Check other splits
print('Churn in 70% train:', y_train_70.mean().round(3))
print('Churn in 30% test:', y_test_30.mean().round(3))
print('Churn in 90% train:', y_train_90.mean().round(3))
print('Churn in 10% test:', y_test_10.mean().round(3))

# Apply SMOTE to address class imbalance on training sets only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

# from imblearn.combine import SMOTEENN
# smote = SMOTEENN(random_state=42)

X_train_80_bal, y_train_80_bal = smote.fit_resample(X_train_80, y_train_80)
X_train_70_bal, y_train_70_bal = smote.fit_resample(X_train_70, y_train_70)
X_train_90_bal, y_train_90_bal = smote.fit_resample(X_train_90, y_train_90)

# X_train_80_bal, y_train_80_bal = X_train_80, y_train_80
# X_train_70_bal, y_train_70_bal = X_train_70, y_train_70
# X_train_90_bal, y_train_90_bal = X_train_90, y_train_90


# Verify balance
print('Balanced 80/20 train:', y_train_80_bal.mean().round(3))
print('Balanced 70/30 train:', y_train_70_bal.mean().round(3))
print('Balanced 90/10 train:', y_train_90_bal.mean().round(3))

## 4.b) Classical ML Algorithms & Cross-Validation
- We construct and tune four classifiers, each with a custom cross-validation grid and justification.
- For each, we perform GridSearchCV cross-validation, select optimal parameters, and explain feature/model selection.
- **Justification**: Each model type offers different decision boundaries and interpretability. This comparison offers insight into which feature transformations are most suitable and which algorithms generalize best on this data.

#### 1) Logistic Regression
- Interpretable, suitable for binary classification.
- Sensitive to correlated and unscaled features—scaling handled above.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

logreg = LogisticRegression(solver='liblinear', max_iter=500)
# Tune penalty and regularization strength
param_grid_lr = {'penalty': ['l1', 'l2'], 'C': [35, 40, 55]}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search for optimal hyperparameters on 80/20 split
gs_lr = GridSearchCV(logreg, param_grid_lr, cv=cv, scoring='f1', n_jobs=-1)
gs_lr.fit(X_train_80_bal, y_train_80_bal)
print(f"Best LR params: {gs_lr.best_params_}; best CV ROC AUC: {gs_lr.best_score_:.3f}")

# Predict on test
y_pred_lr = gs_lr.predict(X_test_20)
y_proba_lr = gs_lr.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_lr))
print("Precision:", precision_score(y_test_20, y_pred_lr))
print("Recall:", recall_score(y_test_20, y_pred_lr))
print("F1 Score:", f1_score(y_test_20, y_pred_lr))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_lr))


#### Justification: Logistic Regression Tuning
- Penalty (l1/l2) tests regularization—l2 for classic Ridge penalty, l1 for sparsity/feature selection benefit.
- C controls inverse strength of regularization (smaller = more regularized; grid covers wide range).
- Stratified 5-fold CV preferred (balances class distribution, gives robust metric estimates).

#### 2) Decision Tree Classifier
- Flexible, interpretable for business rules.
- Hyperparameters control overfitting (max_depth, min_samples_split).

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
param_grid_dt = {
    'max_depth': [2, 4, 6, 8, 12],
    'min_samples_split': [2, 10, 20, 50]
}
gs_dt = GridSearchCV(dt, param_grid_dt, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_dt.fit(X_train_80_bal, y_train_80_bal)
print(f"Best Decision Tree params: {gs_dt.best_params_}; best CV ROC AUC: {gs_dt.best_score_:.3f}")
y_pred_dt = gs_dt.predict(X_test_20)
y_proba_dt = gs_dt.predict_proba(X_test_20)[:,1]

#### Justification: Decision Tree Tuning
- `max_depth` prevents overly complex trees (risk of overfit).
- `min_samples_split` prevents splits on tiny samples that overfit noise.
- Values are chosen to allow both shallow and deep trees for comparison.

#### 3) K-Nearest Neighbors Classifier
- Intuitive, non-parametric, sensitive to feature scaling (handled above).
- Primary tuning: n_neighbors (number of neighbors), weights, distance metric.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'euclidean', 'manhattan']
}
gs_knn = GridSearchCV(knn, param_grid_knn, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_knn.fit(X_train_80_bal, y_train_80_bal)
print(f"Best KNN params: {gs_knn.best_params_}; best CV ROC AUC: {gs_knn.best_score_:.3f}")
y_pred_knn = gs_knn.predict(X_test_20)
y_proba_knn = gs_knn.predict_proba(X_test_20)[:,1]

#### Justification: KNN Tuning
- `n_neighbors`: balances bias-variance (small = highly flexible, large = smoother)
- `weights`: uniform for classic, distance for soft-voting
- `metric`: explores different distance measures (Minkowski covers Euclidean and Manhattan)
- Scaling/standardization is especially crucial, handled above.

#### 4) Ensemble Method: Random Forest Classifier
- Robust, handles both categorical and numerical, reduces variance.
- Primary tuning: number of trees (n_estimators), max_depth.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 8, 12, None],
    'min_samples_split': [2, 10, 20]
}
gs_rf = GridSearchCV(rf, param_grid_rf, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_rf.fit(X_train_80_bal, y_train_80_bal)
print(f"Best RF params: {gs_rf.best_params_}; best CV ROC AUC: {gs_rf.best_score_:.3f}")
y_pred_rf = gs_rf.predict(X_test_20)
y_proba_rf = gs_rf.predict_proba(X_test_20)[:,1]

#### Justification: Random Forest Tuning
- `n_estimators`: More trees = greater stability, but with diminishing returns after around 100-200.
- `max_depth`: Controls tree size/complexity. None = unpruned.
- `min_samples_split`: Prevents overfitting to small branches.
- RF is robust to uninformative features due to built-in selection, so we use the whole set post-encoding.

---
# Section 5: Performance Evaluation & Selection of the Best Model

Now we will evaluate all models using classical metrics for binary classification—**Accuracy, Precision, Recall, F1-Score**, and **ROC-AUC**—on the held-out test set.

### Why these metrics?
- **Accuracy:** Simple overall correctness; less reliable for imbalanced classes.
- **Precision/Recall/F1:** Handle class imbalance, reflect business concern about false approvals (precision) and missed at-risk customers (recall).
- **ROC-AUC:** Comprehensive—measures model's ability to rank churners above non-churners regardless of threshold; best for business action ranking.
- **Justification:** Using all together ensures robust selection; ROC-AUC is often considered most reliable for churn, F1 balances miss and cost.

In [ ]:
# Calculate all metrics for test set (80/20)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

def eval_metrics(y_true, y_pred, y_proba, name):
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_proba)
    }

results = []
results.append(eval_metrics(y_test_20, y_pred_lr, y_proba_lr, 'Logistic Regression'))
results.append(eval_metrics(y_test_20, y_pred_dt, y_proba_dt, 'Decision Tree'))
results.append(eval_metrics(y_test_20, y_pred_knn, y_proba_knn, 'KNN'))
results.append(eval_metrics(y_test_20, y_pred_rf, y_proba_rf, 'Random Forest'))

import pandas as pd
perf_df = pd.DataFrame(results).set_index('Model')
display(perf_df)

## Visualization: Comparison of Evaluation Metrics Across Models

**Justification:**
A bar/chart plot allows for immediate visual benchmarking across all performance metrics for each model. This is the standard for model comparison and helps communicate results to technical and non-technical stakeholders.

In [ ]:
# Plot performance metrics for all models
plt.figure(figsize=(10,6))
perf_df[['Accuracy','Precision','Recall','F1','ROC_AUC']].plot(kind='bar', ax=plt.gca(), rot=15)
plt.title('Comparison of Model Performance Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

## ROC Curves for All Models

Visualizing ROC-AUC curves side by side gives a nuanced view of true positive/false positive trade-offs and allows comparison at every threshold.

In [ ]:
plt.figure(figsize=(8,6))
models_pred = [
    ('Logistic Regression', y_proba_lr),
    ('Decision Tree', y_proba_dt),
    ('KNN', y_proba_knn),
    ('Random Forest', y_proba_rf)
]
for name, yproba in models_pred:
    fpr, tpr, _ = roc_curve(y_test_20, yproba)
    plt.plot(fpr, tpr, label=f'{name} (AUC: {roc_auc_score(y_test_20, yproba):.2f})')
plt.plot([0,1],[0,1],'k--', lw=0.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.tight_layout()
plt.show()

## 5.d) Which Model is Best? (and Why)

**Best Model Selection (with Justification)**

- Model selection should be based primarily on ROC-AUC and F1-score, as they provide robust indicators of balance between generalization and actionable predictive value on imbalanced classification problems like churn.
- From the metrics and ROC curve above, the model with the highest AUC and F1, while maintaining strong precision/recall, should be chosen. In customer churn, ROC-AUC >0.84 is considered highly effective.

**Justification:**
- **Random Forest** usually outperforms others here, due to its ability to model nonlinear relationships and handle interactions between features robustly and without the need for explicit transformation or pruning.
- Logistic Regression offers interpretability, but is outperformed on nonlinear interaction data.
- Decision Tree is interpretable but often overfits or underfits compared to ensembles, as shown by lower test AUC/F1.
- KNN, while useful for simple separation, performs worse on mixed/complex categorical+numeric features and high dimensionality after encoding.

**Conclusion:**
> _The best classical machine learning model for this Telco Customer Churn prediction task is the **Random Forest classifier**, as evidenced by the highest ROC-AUC, F1, and balanced metrics in both visual and tabular comparisons above._

Random Forest should be deployed for customer churn scoring but logistic regression coefficients may still be referenced for business explainability of individual attributes.